In [26]:
# libaries
import matplotlib.pyplot as plt
import seaborn as sns   
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import requests
from pprint import pprint
import json

In [9]:
# import variables from .env file
load_dotenv()

# access the API key
api_key = os.getenv("API_KEY")

In [10]:
# check if key is in application
if api_key is not None:
    print(f"API exists!")
else:
    print("API not found :((")

API exists!


In [17]:
# this is how we use the cURL to call the API in python
response = requests.get(
    "https://api.openaq.org/v3/locations",
    headers={
        "X-API-Key": api_key,
        "Accept": "application/json"
    },
    params={
        "coordinates": "40.7128,-74.0060",
        "radius": 25000}
)

Get the location of new york city within bounds
bbox=west,south,east,north

In [ ]:
data = response.json() # parse content into json

print(data.keys()) # returns [meta, results]
results = data["results"]

dict_keys(['meta', 'results'])
[{'bounds': [-73.9481, 40.8197, -73.9481, 40.8197],
  'coordinates': {'latitude': 40.8197, 'longitude': -73.9481},
  'country': {'code': 'US', 'id': 155, 'name': 'United States'},
  'datetimeFirst': {'local': '2016-03-12T04:00:00-05:00',
                    'utc': '2016-03-12T09:00:00Z'},
  'datetimeLast': {'local': '2026-02-22T17:00:00-05:00',
                   'utc': '2026-02-22T22:00:00Z'},
  'distance': 12838.35149267,
  'id': 384,
  'instruments': [{'id': 2, 'name': 'Government Monitor'}],
  'isMobile': False,
  'isMonitor': True,
  'licenses': [{'attribution': {'name': 'Unknown Governmental Organization',
                                'url': None},
                'dateFrom': '2016-01-30',
                'dateTo': None,
                'id': 33,
                'name': 'US Public Domain'}],
  'locality': 'New York-Northern New Jersey-Long Island',
  'name': 'CCNY',
  'owner': {'id': 4, 'name': 'Unknown Governmental Organization'},
  'provider': 

In [ ]:
# print out all locations from results
print("Total locations: ", len(results))


# iterate thru all locations to see the range of data each one has
for loc in results:
    # find the PM2.5 sensors going through all the sensors for each location
    pm25_sensors = [s for s in loc['sensors'] if s['parameter']['name'] == 'pm25']
    
    if pm25_sensors:
        print(f"{loc['name']}: has {len(pm25_sensors)} PM2.5 sensor(s)")
        print(f"\tTime Rnage: {loc['datetimeFirst']['utc']} to {loc['datetimeLast']['utc']}")
        print(f"\tSensor ID: {pm25_sensors[0]['id']}")
    

Total locations:  60
CCNY: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-12T09:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 673
Manhattan/IS143: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 1097
Bronx - IS52: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 1102
Maspeth: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 1103
Queens: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 1152
PS 19: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2017-12-18T10:00:00Z
	Sensor ID: 1121
Bklyn - PS 314: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 1128
Division Street: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2020-02-19T16:00:00Z
	Sensor ID: 1143
Bklyn - PS274: has 1 PM2.5 sensor(s)
	Time Rnage: 2016-03-10T08:00:00Z to 2026-02-22T22:00:00Z
	Sensor ID: 114

In [59]:
# get CCNY json from the results
# Sensor id is: Sensor ID: 673
ccny_loc = [loc for loc in results if loc['name'] == 'CCNY']

In [89]:
# fetch a new response call with only using that sensor
sensorResponse = requests.get(
    "https://api.openaq.org/v3/sensors/673",
    headers={
        "X-API-Key": api_key,
        "Accept": "application/json"
    }
)

In [90]:
# Here is when I realized that I could access the entire thing with just doing [0]
sensorData = sensorResponse.json()
sensorResults = sensorData['results'][0]

# see the keys with its values
for key, value in sensorResults.items():
    print(key, ":", value)
    
    
#print all the values
#print(json.dumps(sensorResults, indent=4))
pprint(sensorResults["summary"])

id : 673
name : pm25 µg/m³
parameter : {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}
datetimeFirst : {'utc': '2016-03-12T09:00:00Z', 'local': '2016-03-12T04:00:00-05:00'}
datetimeLast : {'utc': '2026-02-22T23:00:00Z', 'local': '2026-02-22T18:00:00-05:00'}
coverage : {'expectedCount': 1, 'expectedInterval': '01:00:00', 'observedCount': 56214, 'observedInterval': '56214:00:00', 'percentComplete': 5621400.0, 'percentCoverage': 5621400.0, 'datetimeFrom': {'utc': '2016-03-12T09:00:00Z', 'local': '2016-03-12T04:00:00-05:00'}, 'datetimeTo': {'utc': '2026-02-22T23:00:00Z', 'local': '2026-02-22T18:00:00-05:00'}}
latest : {'datetime': {'utc': '2026-02-22T23:00:00Z', 'local': '2026-02-22T18:00:00-05:00'}, 'value': 4.5, 'coordinates': {'latitude': 40.8197, 'longitude': -73.9481}}
summary : {'min': -4.9, 'q02': None, 'q25': None, 'median': None, 'q75': None, 'q98': None, 'max': 151.5, 'avg': 7.064731261761605, 'sd': None}
{'avg': 7.064731261761605,
 'max': 151.5,
 'median': No

**Why PM2.5 AQI?**  
Particles less than 2.5 micrometers penetrate deep into lungs and enter the bloodstream, causing issues like asthma, bronchitis, and heart disease.

**Scale for PM2.5 AQI:**  
0-50 (Good): Air quality is satisfactory  
51-100 (Moderate): Acceptable, but potential risk for some, especially those with sensitivities  
101-150 (Unhealthy for Sensitive Groups): General public not likely affected, but sensitive groups are  
151-200 (Unhealthy): Everyone may begin to experience health effects  
201+ (Very Unhealthy/Hazardous): Health alert, serious health effects for all  

**Pagination**
API lets us by using page=1

In [206]:
# now get the measurements by day using sensor ID
dailyResponse = requests.get(
    "https://api.openaq.org/v3/sensors/673/days",
    headers={
        "X-API-Key": api_key,
        "Accept": "application/json"
    },
    params={
        "date_from": "2025-11-22",  # 90 days ago
        "date_to": "2026-02-20",
    }
)

In [ ]:
# now get the results by average per day to print & check data range
dailyData = dailyResponse.json()
dailyResult = dailyData['results']

print(f"Got {len(dailyResult)} days of data")
print(f"From {dailyResult[-1]['period']['datetimeFrom']['utc'][:10]} to {dailyResult[0]['period']['datetimeFrom']['utc'][:10]}")

#pprint(dailyResult)

Got 79 days of data
From 2026-02-20 to 2025-11-30


In [201]:
# extract what we need from daily
data = []

for obs in dailyResult:
    data.append({
        'date': obs['period']['datetimeFrom']['utc'],
        'pm25': obs['value'],
        'coverage': obs['coverage']['percentComplete']
    })

df = pd.DataFrame(data)

In [ ]:
# convert date to datetime
df['date'] = pd.to_datetime(df['date']).dt.date

print(df.shape)
print(df.head(5))
print(df.tail(3))

# count how many days do we have below 75% coverage
count = (df['coverage'] < 75).sum()
print(f"days below 75%: {count}")

(79, 3)
         date   pm25  coverage
0  2025-11-30  10.30      38.0
1  2025-12-01   3.39     100.0
2  2025-12-02   4.86     100.0
3  2025-12-03   4.73     100.0
4  2025-12-04   8.73     100.0
          date   pm25  coverage
76  2026-02-18  29.40     100.0
77  2026-02-19   8.56     100.0
78  2026-02-20   5.18     100.0
total days: 79
days below 75%: 3


In [212]:
# exclude days that have coverage below 50 percent
df = df[df['coverage'] >= 50]
print(df.shape)

(77, 3)


In [213]:
# save df as a dataframe
df.to_csv('dailyMeasurements.csv')